In [ ]:
# Install dependencies (optional)
!pip install tensorflow tensorflow-datasets --quiet

# Import Libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

print("TensorFlow Version:", tf.__version__)


TensorFlow Version: 2.18.0


### 📥 Load Dataset (Cats vs Dogs)


In [ ]:
# Load dataset with TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load(
    "cats_vs_dogs",
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True,
)

# Resize images
IMG_SIZE = (128, 128)

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

train = ds_train.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
test = ds_test.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.09K7VL_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


### 1️⃣ Model without Deep Learning (Sklearn + Random Forest)

In [ ]:
# Convert small sample for sklearn
X = []
y = []

for image, label in ds_train.take(1000):
    image = tf.image.resize(image, IMG_SIZE)
    X.append(image.numpy().flatten())
    y.append(label.numpy())

X = np.array(X)
y = np.array(y)

# Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Model
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print(f"Random Forest Accuracy: {acc*100:.2f}%")


Random Forest Accuracy: 67.00%


### 2 - CNN from Scratch

In [ ]:
cnn_model = models.Sequential([
    layers.Input(shape=(*IMG_SIZE, 3)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

cnn_model.summary()

cnn_model.fit(train, epochs=5, validation_data=test)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 51s 78ms/step - accuracy: 0.6210 - loss: 0.6368 - val_accuracy: 0.7326 - val_loss: 0.5292
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - accuracy: 0.7613 - loss: 0.4970 - val_accuracy: 0.7829 - val_loss: 0.4561
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - accuracy: 0.8074 - loss: 0.4187 - val_accuracy: 0.7904 - val_loss: 0.4415
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 26s 44ms/step - accuracy: 0.8410 - loss: 0.3567 - val_accuracy: 0.8156 - val_loss: 0.4147
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.8750 - loss: 0.2863 - val_accuracy: 0.8089 - val_loss: 0.4764


### 3 - Transfer Learning (EfficientNet Feature Extraction)

In [ ]:
# Load base model
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze

# Create model
transfer_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

transfer_model.compile(optimizer='adam',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])

transfer_model.summary()

transfer_model.fit(train, epochs=5, validation_data=test)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 4, 4, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 67s 82ms/step - accuracy: 0.5048 - loss: 0.6974 - val_accuracy: 0.5099 - val_loss: 0.6931
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 53s 47ms/step - accuracy: 0.5047 - loss: 0.6967 - val_accuracy: 0.5099 - val_loss: 0.6927
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 32s 55ms/step - accuracy: 0.5061 - loss: 0.6963 - val_accuracy: 0.5099 - val_loss: 0.6922
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 46ms/step - accuracy: 0.5103 - loss: 0.6959 - val_accuracy: 0.5099 - val_loss: 0.6918
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - accuracy: 0.5068 - loss: 0.6963 - val_accuracy: 0.5099 - val_loss: 0.6916


### 4 - Fine-Tuning (Unfreeze top 20% of EfficientNet)

In [ ]:
# Unfreeze top 20%
num_layers = len(base_model.layers)
num_to_unfreeze = int(num_layers * 0.2)

for layer in base_model.layers[-num_to_unfreeze:]:
    layer.trainable = True

# Compile with lower learning rate
fine_tune_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

fine_tune_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                         loss='binary_crossentropy',
                         metrics=['accuracy'])

fine_tune_model.summary()

fine_tune_model.fit(train, epochs=5, validation_data=test)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 4, 4, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 2,306,577 (8.80 MB)

 Non-trainable params: 1,744,275 (6.65 MB)

Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 80s 86ms/step - accuracy: 0.5297 - loss: 0.6959 - val_accuracy: 0.4901 - val_loss: 0.7135
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - accuracy: 0.5437 - loss: 0.6860 - val_accuracy: 0.4903 - val_loss: 0.7208
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 29s 50ms/step - accuracy: 0.5509 - loss: 0.6823 - val_accuracy: 0.4938 - val_loss: 0.7116
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 40s 48ms/step - accuracy: 0.5674 - loss: 0.6732 - val_accuracy: 0.4933 - val_loss: 0.7185
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 42s 49ms/step - accuracy: 0.5696 - loss: 0.6727 - val_accuracy: 0.4976 - val_loss: 0.7166
